# Homework 1

## a- Read the dataset file "Cancer.csv" and store it in a Pandas DataFrame.

In [1]:
# Import Pandas
import pandas as pd

In [2]:
# Creating an empty dataframe
cancer_df = pd.DataFrame()

# Reading a CSV file from the web
cancer_df = pd.read_csv('https://raw.githubusercontent.com/ellipsclamation/cs5661_data_science/master/Datasets/Cancer_small.csv')

print(cancer_df)

     Clump_Thickness  Uniformity_of_Cell_Size  Uniformity_of_Cell_Shape  \
0                  5                        1                         1   
1                  5                        4                         4   
2                  3                        1                         1   
3                  6                        8                         8   
4                  4                        1                         1   
5                  8                       10                        10   
6                  1                        1                         1   
7                  2                        1                         2   
8                  2                        1                         1   
9                  4                        2                         1   
10                 5                        3                         3   
11                 8                        7                         5   
12                 7     

## b- Use sklearn functions to split the dataset into testing and training sets.

In [3]:
# Creating the Feature Matrix for cancer dataset:

# create a python list of feature names that would like to pick from the dataset:
feature_cols = ['Clump_Thickness', 'Uniformity_of_Cell_Size', 'Uniformity_of_Cell_Shape',
                'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
                'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses']

# use the above list to select the features from the original DataFrame
X = cancer_df[feature_cols]

# print the first 5 rows
print(X[0:5])

   Clump_Thickness  Uniformity_of_Cell_Size  Uniformity_of_Cell_Shape  \
0                5                        1                         1   
1                5                        4                         4   
2                3                        1                         1   
3                6                        8                         8   
4                4                        1                         1   

   Marginal_Adhesion  Single_Epithelial_Cell_Size  Bare_Nuclei  \
0                  1                            2            1   
1                  5                            7           10   
2                  1                            2            2   
3                  1                            3            4   
4                  3                            2            1   

   Bland_Chromatin  Normal_Nucleoli  Mitoses  
0                3                1        1  
1                3                2        1  
2                3     

In [4]:
# Creating Label column
y = cancer_df['Malignant_Cancer']

# Test y
y[::10]

0      0
10     1
20     1
30     1
40     1
50     0
60     0
70     0
80     1
90     0
100    1
110    1
120    1
130    1
140    0
Name: Malignant_Cancer, dtype: int64

In [5]:
# Picks 30% of data samples for testing set and 70% for training set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

print(X_test)
print('\n')
print(y_test)

     Clump_Thickness  Uniformity_of_Cell_Size  Uniformity_of_Cell_Shape  \
6                  1                        1                         1   
3                  6                        8                         8   
113                5                        6                         7   
12                 7                        4                         6   
24                 5                        6                         5   
129                5                        8                         8   
25                10                       10                        10   
108                7                        2                         4   
128                6                        1                         1   
45                10                        4                         2   
48                 8                        3                         8   
42                 9                       10                        10   
35                10     

## c- Use "Decision Tree Classifier" to predict Cancer based on the training/testing datasets. Then, calculate and report the accuracy of your classifier.

### Decision Tree

In [6]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# "my_decisiontree" is instantiated as an "object" of DecisionTreeCLassifier "class"

my_decisiontree = DecisionTreeClassifier(random_state=2)

### Training on training set:

In [7]:
# Training only on the training set using the method "fit" 
# of the object along with training dataset and labels to train the model.
my_decisiontree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=2,
            splitter='best')

### Testing on testing set:

In [8]:
# Testing on the testing set:
y_predict_dt = my_decisiontree.predict(X_test)

print(y_predict_dt)

[0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0
 1 1 0 1 1 0 1 1]


### Decision Tree Accuracy Evaluation:

In [9]:
# Importing accuracy_score
from sklearn.metrics import accuracy_score

accuracy_dt = accuracy_score(y_test, y_predict_dt)

print(accuracy_dt)

0.866666666667


## d- Now, we want to perform "Bagging"

In [10]:
# Importing resample
from sklearn.utils import resample

bootstrap = []

n_classifiers = 19
bootstrap_size = int(0.8*(len(cancer_df)))

# Perform bootstrapping
for i in range(n_classifiers):
    resample_X_train = resample(X_train, n_samples = bootstrap_size, random_state=i, replace=True)
    resample_y_train = resample(y_train, n_samples = bootstrap_size, random_state=i, replace=True)
    
    Base_DecisionTree = DecisionTreeClassifier(random_state=2)
    Base_DecisionTree.fit(resample_X_train, resample_y_train)
    
    y_predict_dt = Base_DecisionTree.predict(X_test)
    
    bootstrap.append(y_predict_dt)

### Voting

In [11]:
y_predict_vote = []

# Transpose bootstrap list
t_bootstrap = [list(i) for i in zip(*bootstrap)]

# Majority voting
for row in t_bootstrap:
    count0 = 0
    count1 = 0
    
    for y in row:
        if y == 0:
            count0 += 1
        elif y == 1:
            count1 += 1
    
    if count0 >= count1:
        y_predict_vote.append(0)
    else:
        y_predict_vote.append(1)
        
print(y_predict_vote)

[0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1]


### Voting accuracy evaluation:

In [12]:
accuracy_vote = accuracy_score(y_test, y_predict_vote)

print(accuracy_vote)

0.822222222222


## e- Use scikit-learn "Adaboost" classifier to predict Cancer based on the training/testing datasets that you built in part (b). Then, calculate and report the accuracy of your classifier.

### AdaBoost

In [13]:
# Importing AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

my_adaboost = AdaBoostClassifier(n_estimators=19, random_state=2)

### Training on training set:

In [14]:
# Training only on the training set using the method "fit" 
# of the object along with training dataset and labels to train the model.
my_adaboost.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=19, random_state=2)

### Testing on testing set:

In [15]:
# Testing on the testing set:
y_predict_ab = my_adaboost.predict(X_test)

print(y_predict_ab)

[0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 1
 1 1 0 1 1 0 1 1]


### AdaBoost accuracy evaluation:

In [16]:
accuracy_ab = accuracy_score(y_test, y_predict_ab)

print(accuracy_ab)

0.933333333333


## f- Use scikit-learn "Random Forest" classifier to predict Cancer based on the training/testing datasets that you built in part (b). Then, calculate and report the accuracy of your classifier.

### Random Forest

In [17]:
# Importing RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

my_randomforest = RandomForestClassifier(n_estimators=19, bootstrap=True, random_state=2)

### Training on training set:

In [18]:
# Training only on the training set using the method "fit" 
# of the object along with training dataset and labels to train the model.
my_randomforest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=19, n_jobs=1,
            oob_score=False, random_state=2, verbose=0, warm_start=False)

### Testing on testing set:

In [19]:
# Testing on the testing set:
y_predict_rf = my_randomforest.predict(X_test)

print(y_predict_rf)

[0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 1 1
 1 1 0 1 1 0 1 1]


### RandomForest accuracy evaluation:

In [20]:
accuracy_rf = accuracy_score(y_test, y_predict_rf)

print(accuracy_rf)

0.955555555556
